In [ ]:
import torch
import numpy as np
import math

#import pykeops
#pykeops.clean_pykeops()

In [ ]:
device = torch.device("cuda")

In [ ]:
from gechebnet.graph.graph import SE2GEGraph

In [ ]:
xi, eps = 1., 1.
se2_graph = SE2GEGraph(
    nx=30,
    ny=30,
    ntheta=12,
    knn=16,
    sigmas=(xi / eps, xi, 1.0),
    weight_kernel=lambda sqdistc, sigmac: torch.exp(-sqdistc / sigmac),
    device=torch.device("cuda")
)

# Data

In [ ]:
x = torch.rand(16, 3, 30*30*12, device=device)

## ChebNet

In [ ]:
from gechebnet.model.chebnet import GEChebNet

In [ ]:
K = 5
in_channels = 3
out_channels = 10
hidden_channels = [16, 16, 32, 64]
pooling = "max"

In [ ]:
model = GEChebNet(se2_graph, K, in_channels, hidden_channels, out_channels, pooling, device=device)
model = model.to(device)
model

In [ ]:
with torch.no_grad():
    y = model(x)
y

In [ ]:
model.sparsify_laplacian("edges", 0.5)
model.norm_laplacian

In [ ]:
with torch.no_grad():
    y = model(x)
y

In [ ]:
model.sparsify_laplacian("nodes", 0.9)
model.norm_laplacian

In [ ]:
with torch.no_grad():
    y = model(x)
y

## ResChebNet

In [ ]:
from gechebnet.model.reschebnet import ResGEChebNet

In [ ]:
K = 10
in_channels = 3
out_channels = 10
hidden_channels = [[16, 16, 16], [16, 32, 32], [32, 64, 64]]

In [ ]:
model = ResGEChebNet(se2_graph, K, in_channels, hidden_channels, out_channels, device=device)
model = model.to(device)
model

In [ ]:
with torch.no_grad():
    y = model(x)
y

In [ ]:
model.sparsify_laplacian("edges", 0.5)
model.norm_laplacian

In [ ]:
with torch.no_grad():
    y = model(x)
y

In [ ]:
model.sparsify_laplacian("nodes", 0.9)
model.norm_laplacian

In [ ]:
with torch.no_grad():
    y = model(x)
y